# global average pooling

*출처 : <https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/> *


- CNN구조의 FC레이어는 파라미터의 수가 많아 overfitting의 문제가 있으나, FC 레이어 대신 GAP를 사용한다면, 과적합을 막을 수 있다. 또한, 별도의 처리(image annotation)를 하지 않아도 image localization이 가능하다.
----------------------------------

- 보통 CNN구조는 아래의 vgg-16같이 convolution 레이어 여러 개와 pooling 레이어가 반복되는 구조이다. 그리고 마지막에 FC 레이어와 softmax로 사물의 class를 판별한다.

![vgg-16](./image/vgg-16.png)

- 마지막에 오는 max pooling 레이어는 평평하게 펴지는 FC레이어에 연결된다. 모델의 대부분의 파라미터는 FC레이어에 속하게 된다.

- 이러한 모델 구조는 `오버피팅`의 위험성을 가진다. 실제로는 이를 막기 위해 dropout 레이어를 사용한다.

--------------------------

### Global Average Pooling

max pooling 레이어랑 비슷하게, `GAP`(global average pooling) 레이어는 공간적 차원을 줄이기 위해 사용한다. 밑의 그림을 예로 보면, GAP 는 h w값의 평균을 사용해 h x w x d 차원을 1 x 1 x d 차원으로 줄인다. 

![gap](./image/gap.png)

첫 번째로 GAP 레이어를 제안한 논문은, 마지막 max pooling 레이어가 각 이미지 카테고리에 대한 활성화 맵을 포함하는 구조로 설계했다. max pooling 레이어가 GAP레이어의 입력으로 들어가고, 그것은 classification 작업에서 각각 객체가 있을 확률을 가진 단일 항목 벡터를 산출한다. 그리고 각 클래스의 예측 확률을 계산하기 위해 softmax 활성화 함수를 사용했다. 만약 [원문](https://arxiv.org/pdf/1312.4400.pdf)을 본다면, 특히 섹션 3.2의 gap부분을 보는 것을 추천한다. 

> ![gap2](./image/gap2.jpg)
- 분류에서, 마지막 convolutional 레이어의 feature map은 vectorize되고 FC레이어의 입력으로 들어간다. 그 뒤에 softmax logistic regression 레이어가 온다. 그러나, fully connected layer는 오버피팅에 취약하다.
- 기존의 CNN에 있는 FC 레이어를 대체하는 global average pooling은 classification task에서 대응되는 각각의 카테고리에 feature map을 생성한다. feature map의 끝에 FC레이어를 쌓는 대신, 각 feature map의 평균을 취하고, 그 결과 벡터를 바로 softmax layer의 입력으로 넣어준다.
- GAP사용의 첫 번째 장점은, 특징 맵과 카테고리 사이의 대응을 시행해서, convolution구조와 더 잘 어울린다. 그래서 특징 맵이 쉽게 카테고리의 confidence map(신뢰 특징? ground-truth 같은 것인 듯)으로 변환될 수 있다.
- 두 번째 장점은, GAP에는 최적화시킬 파라미터가 존재하지 않아, 이 레이어에서의 오버피팅이 방지된다.
- 더욱이, gap는 공간적인 정보를 합쳐서 내놓기 때문에 input의 공간적인 변형에 더 잘 대응할 수 있다.

ResNet-50 model은 dense 레이어를 전부 없애는 극단적인 방법보다 덜한 것을 채택한다. GAP 레이어 뒤에, softmax 활성화 함수와 함께 object class를 예상해주는 하나의 densely connected layer가 온다.

---------------------

### Object Localization

2016년에, MIT의 research팀은 GAP레이어를 사용한 CNN(GAP-CNN)을 시연하였다. 그 모델은 classification작업 뿐 아니라 `object localization`작업을 위해 train되었다. 즉, GAP-CNN은 별도의 작업 없이도 이미지에 어떤 객체가 포함되어 있는지 뿐만 아니라 객체가 어디에 있는지까지 말해주는 것이다. localization은 heat map(class activation map으로 언급된다)으로 표현되는데, 색상 체계는 GAP-CNN이 object identification 작업을 수행하는 데 상대적으로 중요한 영역을 식별한다.

![gap3](./image/gap3.png)

ResNet-50에서는, 예측된 object 카테고리를 마지막 dense 레이어의 하나의 노드에 대응시켰다. 그리고 이 하나의 노드는 앞의 flatten layer의 모든 노드에 연결된다. w(k)는 Flatten레이어의 k번째 노드와 연결된 weight를 나타내는데, 예측 이미지 카테고리와 대응되는 output노드와 연결되어 있다. 

class activation map을 얻으려면, 우리는 단지 w1 x f1 + w2 x f2 + ..... 이런 합만 계산하면 된다.